In [3]:

"""
Connect senior:Mainland
"""
import pandas as pd
import json
import requests
def ssql(sql):
    admin_token = 'ae810dc4282e3de074ab08ab631f3461218accf99143b8f3a272e64c1b4c6d0e'
    url = 'http://sensor.wb-intra.com/api/sql/query?token=%s&project=production' % admin_token
    data = {'q': sql, 'format': 'json'}
    req = requests.post(url,data)
    req_dec = req.content.decode()
    try:
        req_json = json.loads('[' + req_dec.replace('\n', ',')[:-1] + ']')
        df_d_id = pd.DataFrame(req_json)
        return df_d_id
    except:
        print(req_dec)

In [11]:
df = ssql("""

SELECT t2.date,
       t2.distinct_id,
       t2.game_duration
FROM
  (SELECT t1.date, t1.distinct_id
   FROM
     (SELECT date,distinct_id,
                  appId
      FROM events
      WHERE event = 'register'
        AND date BETWEEN '2020-12-31' AND CURRENT_DATE() - interval 2 DAY
        AND appId IN ('20014',
                      '30015')
      GROUP BY 1,
               2,
               3) t1
   LEFT JOIN
     (SELECT date,distinct_id
      FROM events
      WHERE event = 'advSend'
        AND date BETWEEN '2020-12-31' AND CURRENT_DATE() - interval 2 DAY
        AND adv_id = 'b_killer_newuserlead_201217'
        AND appId IN ('20014',
                      '30015')
        AND RESULT = 'pop-up'
      GROUP BY 1,
               2)t2 ON t1.date = t2.date
   AND t1.distinct_id = t2.distinct_id
   WHERE t2.distinct_id IS NULL
   GROUP BY 1,
            2) t1
JOIN
  (SELECT date,distinct_id,
               sum(duration/60000)AS game_duration
   FROM events
   WHERE event = 'gameOver'
     AND gameTypeId = 1800
     AND date BETWEEN '2021-01-01' AND CURRENT_DATE() - interval 1 DAY
     AND appId IN ('20014',
                   '30015')
   GROUP BY 1,
            2)t2 ON t1.distinct_id = t2.distinct_id
AND t1.date = t2.date
""")

In [12]:
df

,date,distinct_id,game_duration
0,2021-01-05,140152538,4.550
1,2021-01-02,131447769,19.635
2,2021-01-01,124293641,18.721
3,2021-01-05,125302765,31.123
4,2021-01-11,138519199,4.866
...,...,...,...
36685,2021-01-02,136930298,14.857
36686,2021-01-06,127746889,211.043
36687,2021-01-01,148205122,5.557
36688,2021-01-09,141187673,11.032


In [13]:
df.groupby(by = ['date'])['game_duration'].describe().to_clipboard()